<a href="https://colab.research.google.com/github/biaferre/bioinspirada/blob/main/Finding_Primes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Encontrando números primos / Finding primes

> Disciplina de Tópicos Avançados em IA: Algoritmos Bioinspirados | CIn UFPE 2025.1

> Grupo: Beatriz Férre e Ronald Silva

Exercício em Python para encontrar primos dentro de um intervalo de números inteiros com mais de 6 casas decimais. Experimentações com testes de primalidade, tamanho populacional e intervalo da busca, métodos de seleção (torneio x roleta), representações com inteiros e funções de combinação e mutação pra inteiros.

/ Python exercise to find prime numbers within a range of integers with more than 6 digits. Experiments with primality tests, population size and search range, selection methods (tournament vs. roulette), representations using integers, and combination and mutation functions for integers.

### Parâmetros

In [66]:
import random

MIN_VAL = 10**5
MAX_VAL = 10**7
POP_SIZE = 20
GEN = 100

### Código base
Fornecida na monitoria para ser aprimorada nas seções subsequentes.

In [67]:
# Checar se o número é primo
def is_prime(n):
    if n < 2:
        return False
    for i in range(2, int(n**0.5)+1):
        if n % i == 0:
            return False
    return True

# Função de fitness: 1 se primo, senão penaliza pelo número de divisores
def fitness(n):
    if is_prime(n):
        return 1000  # grande valor de fitness
    else:
        divisores = sum(1 for i in range(2, n) if n % i == 0)
        return 1 / (divisores + 1)

# Inicializa a população de inteiros
def init_population(size, min_val, max_val):
    return [random.randint(min_val, max_val) for _ in range(size)]

# Seleção: Torneio binário
def selection(population, fitnesses):
    selected = []
    for _ in range(len(population)):
        if len(population) < 2:
            return population  # evita erro se houver apenas 1 indivíduo
        i, j = random.sample(range(len(population)), 2)
        selected.append(population[i] if fitnesses[i] > fitnesses[j] else population[j])
    return selected

# Crossover: combinação de bits
def crossover(parent1, parent2):
    p1 = format(parent1, 'b')
    p2 = format(parent2, 'b')
    max_len = max(len(p1), len(p2))
    p1 = p1.zfill(max_len)
    p2 = p2.zfill(max_len)
    point = random.randint(1, max_len - 1)
    child_bin = p1[:point] + p2[point:]
    return int(child_bin, 2)

# Mutação: troca aleatória de bits
def mutate(n, mutation_rate=0.05): # aumentar 0.2
    bits = list(format(n, 'b'))
    for i in range(len(bits)):
        if random.random() < mutation_rate:
            bits[i] = '1' if bits[i] == '0' else '0'
    return int(''.join(bits), 2)

# Algoritmo evolutivo
def evolutionary_prime_search(pop_size=10, min_val=10**5, max_val=10**6, generations=100):
    population = init_population(pop_size, min_val, max_val)

    for gen in range(generations):
        fitnesses = [fitness(ind) for ind in population]

        # Melhor indivíduo da geração
        best_idx = fitnesses.index(max(fitnesses))
        best_ind = population[best_idx]
        best_fit = fitnesses[best_idx]

        print(f"Geração {gen}: Melhor fitness = {best_fit:.5f}, Melhor indivíduo = {best_ind}")

        # Verifica se encontrou um primo
        if best_fit == 1000:
            print(f"\n✅ Primo encontrado na geração {gen}: {best_ind}")
            return best_ind

        # Seleção
        selected = selection(population, fitnesses)

        # Crossover com reposição até manter o tamanho original
        next_population = []
        while len(next_population) < pop_size:
            p1, p2 = random.sample(selected, 2)
            child = crossover(p1, p2)
            next_population.append(child)

        # Mutação
        population = [mutate(ind) for ind in next_population]

    print("\n❌ Nenhum primo encontrado após todas as gerações.")
    return None

# Executa o algoritmo
evolutionary_prime_search()

Geração 0: Melhor fitness = 0.33333, Melhor indivíduo = 904018
Geração 1: Melhor fitness = 0.33333, Melhor indivíduo = 969554
Geração 2: Melhor fitness = 1000.00000, Melhor indivíduo = 674083

✅ Primo encontrado na geração 2: 674083


674083

### Experimentações

1. Teste de primalidade e definição de fitness
O teste do código base de primalidade consiste apenas em iterar por todos os valores de 2 a raiz de n e ver se há algum divisor de n. É um teste pouco robusto e custoso. Do mesmo modo, seu fitness é definido com base em quantos divisores um número não primo tem, retornando 1000 se for primo.

In [132]:
# Primalidade
def is_prime(n):
    if n < 2:
        return False
    for i in range(2, int(n**0.5)+1):
        if n % i == 0:
            return False
    return True

# Função de fitness: 1 se primo, senão penaliza pelo número de divisores
def fitness(n):
    if is_prime(n):
        return 1000 
    else:
        divisores = sum(1 for i in range(2, n) if n % i == 0)
        return 1 / (divisores + 1)

Uma primeira experimentação que fizemos foi com Teoria de Fermat. Pra isso, usamos exponenciação binária (que é mais eficiente que exponenciação crua).
Contudo, existem dois problemas nessa implementação: 
a) Podem haver compostos que passam o teste (problema pequeno, já que eles são raríssimos)
b) Como nossa implementação é determinística, o fitness fica limitado a 0, se composto, e 1, se primo

O 2o problema é extremamente importante, visto que, como mais pra frente vamos implementar torneio, o total fitness não pode ser 0.

In [118]:
def binpower(base, exp, mod): # Binary exponentiation
    result = 1
    base = base % mod
    while exp > 0:
        if (exp % 2) == 1:  # se exp é ímpar
            result = (result * base) % mod
        exp = exp >> 1  # divide exp por 2
        base = (base * base) % mod
    return result

def probablyPrimeFermat(n, iter=5):
    if (n < 4):
        return n == 2 or n == 3

    for _ in range(iter): 
        a = 2 + random.randint(2, n - 2)
        if (binpower(a, n - 1, n) == 1):
            return 1
    return 0

def fitness(n):
    return probablyPrimeFermat(n)

Uma solução é rodar o teste múltiplas vezes dentro do fitness, já que estamos escolhendo um "a" aleatório toda vez. Quanto mais testes um número passar, maiores as chances de ele ser primo. Reescrevendo:

In [119]:
def probablyPrimeFermat(n):
    if (n < 4):
        return n == 2 or n == 3

    a = random.randint(2, n - 2)
    if (binpower(a, n - 1, n) != 1):
        return 0
    
    return 1

def fitness(n, iter=100):
    if n < 2:
        return 0

    passed = 0
    for _ in range(iter):
        if probablyPrimeFermat(n) == 1:
            passed += 1

    print(passed)
    return passed / iter

2. Inicialização da população

In [ ]:
# Inicialização da população
def init_population(size, min_val, max_val):
    population = []
    while len(population) < size:
      val = random.randint(min_val, max_val)
      if (val % 2 != 0): # apenas ímpares
        population.append(val)
    return population

3. Método de seleção

No código base, tínhamos o método de seleção por torneio. Esse método funciona, mas não é tão robusto e não dá chance de números com baixo fitness, que podem ter características interessantes, serem escolhidos. Por isso implementamos roleta:

In [72]:
# Seleção: Roleta
def selection(population, fitnesses):
    total_fitness = sum(fitnesses)
    individual_probabilities = []
    for i in population:
      i_probability = fitnesses[population.index(i)] / total_fitness
      individual_probabilities.append((i, i_probability))

    print(individual_probabilities)

    p1 = 0
    roulette_num = random.random()
    for i in individual_probabilities:
      roulette_num -= i[1]
      if roulette_num <= 0:
        p1 = i
        break

    p2 = 0
    roulette_num = random.random()
    for i in individual_probabilities:
      roulette_num -= i[1]
      if roulette_num <= 0:
        p2 = i
        break

    return p1, p2

4. Método de crossover

Ao invés de formatarmos o número para uma representação binária para usar métodos de crossover binários, resolvemos manter o número em seu estado de inteiro e implementar um método de crossover que manipula valores inteiros, isto é, a Recombinação Intermediária.

In [ ]:
# Crossover: Recombinação Intermediária
def crossover(p1, p2):
    d = random.uniform(0, 0.5)
    child = int(p1 * d + p2 * (1 - d))
    return child

5. Método de mutação

In [74]:
# Mutação: troca aleatória de bits
def mutate(n, mutation_rate=0.05):
    bits = list(format(n, 'b'))
    for i in range(len(bits)):
        if random.random() < mutation_rate:
            bits[i] = '1' if bits[i] == '0' else '0'
    return int(''.join(bits), 2)

6. Algoritmo evolutivo

O algoritmo evolutivo, adaptado para nossos novos operadores de mutação e crossover, ficou assim:

In [121]:

# Algoritmo evolutivo
def evolutionary_prime_search(pop_size=POP_SIZE, min_val=MIN_VAL, max_val=MAX_VAL, generations=GEN):
    population = init_population(pop_size, min_val, max_val)

    # Adicionando pra poder plotar
    avg_fitnesses = []
    best_fitnesses = []

    for gen in range(generations):
        fitnesses = [fitness(ind) for ind in population]

        avg_fitness = sum(fitnesses) / len(fitnesses)
        best_idx = fitnesses.index(max(fitnesses))
        best_ind = population[best_idx]
        best_fit = fitnesses[best_idx]

        # Atualizando as listas
        avg_fitnesses.append(avg_fitness)
        best_fitnesses.append(best_fit)


        print(f"Geração {gen}: Melhor fitness = {best_fit:.5f}, Melhor indivíduo = {best_ind}")

        if best_fit == 1:
            print(f"\n✅ Primo encontrado na geração {gen}: {best_ind}")
            return avg_fitness, best_fitnesses

        # Seleção
        selected = selection(population, fitnesses)
        print(selected)
        p1 = selected[0][0]
        p2 = selected[1][0]

        # Crossover com reposição até manter o tamanho original
        next_population = []
        while len(next_population) < pop_size:
            child = crossover(p1, p2)
            next_population.append(child)

        # Mutação
        population = [mutate(ind) for ind in next_population]

    print("\n❌ Nenhum primo encontrado após todas as gerações.")
    return avg_fitness, best_fitnesses


In [122]:
# Executa o algoritmo
evolutionary_prime_search()

0
0
100
0
0
0
0
0
100
0
0
0
100
0
0
0
0
0
0
0
Geração 0: Melhor fitness = 1.00000, Melhor indivíduo = 7157039

✅ Primo encontrado na geração 0: 7157039


(0.15, [1.0])

### Análise
Vamos criar métodos pra rodar simulações do algoritmo e visualizar seus resultados, a partir dos arrays que estamos atualizando de best_fitnesses e avg_fitnesses:

In [126]:
def run_simulations(num_simulations):
    all_avg_curves = []
    all_best_curves = []

    for _ in range(num_simulations):
        avg_curve, best_curve = evolutionary_prime_search()
        all_avg_curves.append(avg_curve)
        all_best_curves.append(best_curve)

    return all_avg_curves, all_best_curves

Criando nossos gráficos pra visualizar:

In [128]:
import matplotlib.pyplot as plt
import numpy as np

def plot_avg_curves(avg_curves, best_curves):
    max_generations = max(len(c) for c in avg_curves)

    for i in range(len(avg_curves)):
        last_avg = avg_curves[i][-1]
        last_best = best_curves[i][-1]
        while len(avg_curves[i]) < max_generations:
            avg_curves[i].append(last_avg)
            best_curves[i].append(last_best)

    mean_avg = np.mean(avg_curves, axis=0)
    mean_best = np.mean(best_curves, axis=0)

    plt.figure(figsize=(12, 5))

    plt.subplot(1, 2, 1)
    plt.plot(mean_avg, label='Fitness Médio')
    plt.xlabel("Geração")
    plt.ylabel("Fitness Médio")
    plt.title("Fitness Médio por Geração (média das simulações)")
    plt.grid(True)
    plt.legend()

    plt.subplot(1, 2, 2)
    plt.plot(mean_best, label='Melhor Fitness', color='orange')
    plt.xlabel("Geração")
    plt.ylabel("Melhor Fitness")
    plt.title("Melhor Fitness por Geração (média das simulações)")
    plt.grid(True)
    plt.legend()

    plt.tight_layout()
    plt.show()


Rodando a simulação 1000 vezes:

In [ ]:
num_sim = 10
avg_curves, best_curves = run_simulations(num_sim)
plot_avg_curves(avg_curves, best_curves)

Geração 0: Melhor fitness = 1000.00000, Melhor indivíduo = 3688693
[(1804333, 0.000166350997612517), (3276379, 0.000166350997612517), (3688693, 0.49905299283755095), (4585873, 0.000166350997612517), (9518937, 7.12932846910787e-05), (8171705, 0.000166350997612517), (9756579, 3.3270199522503396e-05), (7190259, 7.12932846910787e-05), (5723559, 4.536845389432281e-05), (7541727, 1.6098483639921e-05), (5779543, 3.3270199522503396e-05), (2202207, 2.16979562103283e-05), (9824891, 0.000166350997612517), (9467117, 0.000166350997612517), (2013053, 0.000166350997612517), (920215, 0.000166350997612517), (9023973, 3.3270199522503396e-05), (5532955, 7.12932846910787e-05), (5424437, 0.000166350997612517), (6745801, 0.49905299283755095)]
((3688693, 0.49905299283755095), (3688693, 0.49905299283755095))
Geração 1: Melhor fitness = 1000.00000, Melhor indivíduo = 3688693
[(3688693, 0.09089252051658717), (3950839, 3.0297506838862385e-05), (3688693, 0.09089252051658717), (3692789, 3.0297506838862385e-05), (3